<a href="https://colab.research.google.com/github/fatemafaria142/Evaluating-LLMs-Generated-Food-Recipes-in-Comparison-to-Established-Reference-Recipes/blob/main/Recipe_Generation_using_TinyLlama.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Install Required Packages**

In [1]:
!pip install accelerate peft bitsandbytes transformers trl datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 49.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.9/78.9 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.5 MB/s eta 0:00:00


In [2]:
from google.colab import drive
# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Load the required packages**

In [3]:
import torch
from datasets import load_dataset, Dataset
from peft import LoraConfig, AutoPeftModelForCausalLM
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments
from trl import SFTTrainer
import os

### **Dataset Link:** https://huggingface.co/datasets/recipe_nlg

In [4]:
import pandas as pd

# Load the original dataset
dataset = pd.read_csv("/content/drive/MyDrive/Recipe Generation/full_dataset.csv")

# Extract the first 3000 rows
subset_dataset = dataset.head(3000)

# Save the subset to a new CSV file
subset_dataset.to_csv("/content/drive/MyDrive/Recipe Generation/recipe_generation.csv", index=False)


In [5]:
import pandas as pd
dataset = pd.read_csv("/content/drive/MyDrive/Recipe Generation/recipe_generation.csv")
dataset.head()

,Unnamed: 0,title,ingredients,directions,link,source,NER
0,0,No-Bake Nut Cookies,"[""1 c. firmly packed brown sugar"", ""1/2 c. eva...","[""In a heavy 2-quart saucepan, mix brown sugar...",www.cookbooks.com/Recipe-Details.aspx?id=44874,Gathered,"[""brown sugar"", ""milk"", ""vanilla"", ""nuts"", ""bu..."
1,1,Jewell Ball'S Chicken,"[""1 small jar chipped beef, cut up"", ""4 boned ...","[""Place chipped beef on bottom of baking dish....",www.cookbooks.com/Recipe-Details.aspx?id=699419,Gathered,"[""beef"", ""chicken breasts"", ""cream of mushroom..."
2,2,Creamy Corn,"[""2 (16 oz.) pkg. frozen corn"", ""1 (8 oz.) pkg...","[""In a slow cooker, combine all ingredients. C...",www.cookbooks.com/Recipe-Details.aspx?id=10570,Gathered,"[""frozen corn"", ""cream cheese"", ""butter"", ""gar..."
3,3,Chicken Funny,"[""1 large whole chicken"", ""2 (10 1/2 oz.) cans...","[""Boil and debone chicken."", ""Put bite size pi...",www.cookbooks.com/Recipe-Details.aspx?id=897570,Gathered,"[""chicken"", ""chicken gravy"", ""cream of mushroo..."
4,4,Reeses Cups(Candy),"[""1 c. peanut butter"", ""3/4 c. graham cracker ...","[""Combine first four ingredients and press in ...",www.cookbooks.com/Recipe-Details.aspx?id=659239,Gathered,"[""peanut butter"", ""graham cracker crumbs"", ""bu..."


In [6]:
length_of_data = len(dataset)
# Print the length
print(length_of_data)

3000


In [7]:
model_id="TinyLlama/TinyLlama-1.1B-Chat-v1.0"
output_model="tinyllama-instructions-v1"

# **Dataset preparation**

In [8]:
def prepare_train_data(data):
    data_df = data[["ingredients", "directions"]].copy()
    template = "Imagine you're a chef, and you've got these delicious ingredients in front of you. Now, all you need are to give the directions on how to cook up a mouthwatering dish."

    data_df["text"] = data_df.apply(lambda x: f"user\n{template}\n input\n{x['ingredients']}\nchef\n{x['directions']}\n", axis=1)
    data = Dataset.from_pandas(data_df)

    return data


In [9]:
data = prepare_train_data(dataset)

In [10]:
data

Dataset({
    features: ['ingredients', 'directions', 'text'],
    num_rows: 3000
})

In [11]:
data[0]

{'ingredients': '["1 c. firmly packed brown sugar", "1/2 c. evaporated milk", "1/2 tsp. vanilla", "1/2 c. broken nuts (pecans)", "2 Tbsp. butter or margarine", "3 1/2 c. bite size shredded rice biscuits"]',
 'directions': '["In a heavy 2-quart saucepan, mix brown sugar, nuts, evaporated milk and butter or margarine.", "Stir over medium heat until mixture bubbles all over top.", "Boil and stir 5 minutes more. Take off heat.", "Stir in vanilla and cereal; mix well.", "Using 2 teaspoons, drop and shape into 30 clusters on wax paper.", "Let stand until firm, about 30 minutes."]',
 'text': 'user\nImagine you\'re a chef, and you\'ve got these delicious ingredients in front of you. Now, all you need are to give the directions on how to cook up a mouthwatering dish.\n input\n["1 c. firmly packed brown sugar", "1/2 c. evaporated milk", "1/2 tsp. vanilla", "1/2 c. broken nuts (pecans)", "2 Tbsp. butter or margarine", "3 1/2 c. bite size shredded rice biscuits"]\nchef\n["In a heavy 2-quart saucep

In [12]:
data[1]

{'ingredients': '["1 small jar chipped beef, cut up", "4 boned chicken breasts", "1 can cream of mushroom soup", "1 carton sour cream"]',
 'directions': '["Place chipped beef on bottom of baking dish.", "Place chicken on top of beef.", "Mix soup and cream together; pour over chicken. Bake, uncovered, at 275\\u00b0 for 3 hours."]',
 'text': 'user\nImagine you\'re a chef, and you\'ve got these delicious ingredients in front of you. Now, all you need are to give the directions on how to cook up a mouthwatering dish.\n input\n["1 small jar chipped beef, cut up", "4 boned chicken breasts", "1 can cream of mushroom soup", "1 carton sour cream"]\nchef\n["Place chipped beef on bottom of baking dish.", "Place chicken on top of beef.", "Mix soup and cream together; pour over chicken. Bake, uncovered, at 275\\u00b0 for 3 hours."]\n'}

## **We have to model the Model (not the base version)**

In [14]:
def get_model_and_tokenizer(mode_id):

    tokenizer = AutoTokenizer.from_pretrained(mode_id)
    tokenizer.pad_token = tokenizer.eos_token
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype="float16", bnb_4bit_use_double_quant=True
    )
    model = AutoModelForCausalLM.from_pretrained(
        mode_id, quantization_config=bnb_config, device_map="auto"
    )
    model.config.use_cache=False
    model.config.pretraining_tp=1
    return model, tokenizer

In [15]:
model, tokenizer = get_model_and_tokenizer(model_id)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

# **Setting up the LoRA**

In [16]:
peft_config = LoraConfig(
        r=8, lora_alpha=16, lora_dropout=0.05, bias="none", task_type="CAUSAL_LM"
    )

# **Training Arguments**

In [17]:
training_arguments = TrainingArguments(
        output_dir=output_model,
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        optim="paged_adamw_32bit",
        learning_rate=2e-4,
        lr_scheduler_type="cosine",
        save_strategy="epoch",
        logging_steps=10,
        num_train_epochs=10,
        max_steps=350,
        fp16=True,
        # push_to_hub=True
    )

In [18]:
trainer = SFTTrainer(
        model=model,
        train_dataset=data,
        peft_config=peft_config,
        dataset_text_field="text",
        args=training_arguments,
        tokenizer=tokenizer,
        packing=False,
        max_seq_length=1024
    )

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

# **Training starts here**

In [19]:
trainer.train()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
10,1.772700
20,1.503500
30,1.296300
40,1.166400
50,1.024800
60,0.976400
70,0.953900
80,0.991100
90,0.966700
100,0.920000


TrainOutput(global_step=350, training_loss=0.9876771572657994, metrics={'train_runtime': 335.4072, 'train_samples_per_second': 4.174, 'train_steps_per_second': 1.044, 'total_flos': 1900892203978752.0, 'train_loss': 0.9876771572657994, 'epoch': 0.47})

# **Merging the LoRA with the base model**

In [21]:
from peft import AutoPeftModelForCausalLM, PeftModel
from transformers import AutoModelForCausalLM
import torch
import os

model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float16, load_in_8bit=False,
                                             device_map="auto",
                                             trust_remote_code=True)

model_path = "/content/tinyllama-instructions-v1/checkpoint-350"

peft_model = PeftModel.from_pretrained(model, model_path, from_transformers=True, device_map="auto")

model = peft_model.merge_and_unload()

In [22]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 2048)
    (layers): ModuleList(
      (0-21): 22 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=5632, bias=False)
          (up_proj): Linear(in_features=2048, out_features=5632, bias=False)
          (down_proj): Linear(in_features=5632, out_features=2048, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (lm_head)

# **Inference from the LLM for many examples**

In [27]:
def formatted_prompt(template, ingredients)-> str:
    return f"<|im_start|>user\n{template}<|im_end|>\n<|im_start|>input\n{ingredients}<|im_end|>\n<|im_start|>chef:"

In [36]:
from transformers import GenerationConfig
from time import perf_counter

def generate_response(instruction, user_input):
    prompt = formatted_prompt(instruction, user_input)

    inputs = tokenizer([prompt], return_tensors="pt")
    generation_config = GenerationConfig(
        penalty_alpha=0.6,
        do_sample=True,
        top_k=5,
        temperature=0.5,
        repetition_penalty=1.2,
        max_new_tokens=512,
        pad_token_id=tokenizer.eos_token_id
    )
    start_time = perf_counter()

    inputs = tokenizer(prompt, return_tensors="pt").to('cuda')

    outputs = model.generate(**inputs, generation_config=generation_config)
    decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()  # Remove leading and trailing whitespaces
    print(decoded_output)
    output_time = perf_counter() - start_time
    print(f"Time taken for inference: {round(output_time, 2)} seconds")

### **Example No:1**

In [37]:
instruction = "Imagine you're a chef, and you've got these delicious ingredients in front of you. Now, all you need are to give the directions on how to cook up a mouthwatering dish."
ingredients = '["1 large whole chicken", "2 (10 1/2 oz.) cans chicken gravy", "1 (10 1/2 oz.) can cream of mushroom soup", "1 (6 oz.) box Stove Top stuffing", "4 oz. shredded cheese"]'
generate_response(instruction=instruction, user_input=ingredients)

<|im_start|>user
Imagine you're a chef, and you've got these delicious ingredients in front of you. Now, all you need are to give the directions on how to cook up a mouthwatering dish.<|im_end|>
<|im_start|>input
["1 large whole chicken", "2 (10 1/2 oz.) cans chicken gravy", "1 (10 1/2 oz.) can cream of mushroom soup", "1 (6 oz.) box Stove Top stuffing", "4 oz. shredded cheese"]<|im_end|>
<|im_start|>chef:
Ingredients for stuffing:
- 3 cups bread crumbs
- 1 egg beaten with milk
- 1 pkg. chopped celery leaves
- 1 pk. minced fresh parsley
- 1 tsp. salt
- 1/8 tsp. pepper

Preheat oven to 350°F. Mix together dry ingredients; set aside. Cut chicken into small pieces. Stuff each piece of chicken with mixture, then roll it tightly in bread crumbs. Place in baking dish. Pour gravy over top. Bake uncovered at 350°F for about 1 hour or until golden brown. Serves 4.
Time taken for inference: 8.66 seconds


### **Example No:2**

In [38]:
instruction = "Imagine you're a chef, and you've got these delicious ingredients in front of you. Now, all you need are to give the directions on how to cook up a mouthwatering dish."
ingredients = '["2 (16 oz.) pkg. frozen corn", "1 (8 oz.) pkg. cream cheese, cubed", "1/3 c. butter, cubed", "1/2 tsp. garlic powder", "1/2 tsp. salt", "1/4 tsp. pepper"]'
generate_response(instruction=instruction, user_input=ingredients)

<|im_start|>user
Imagine you're a chef, and you've got these delicious ingredients in front of you. Now, all you need are to give the directions on how to cook up a mouthwatering dish.<|im_end|>
<|im_start|>input
["2 (16 oz.) pkg. frozen corn", "1 (8 oz.) pkg. cream cheese, cubed", "1/3 c. butter, cubed", "1/2 tsp. garlic powder", "1/2 tsp. salt", "1/4 tsp. pepper"]<|im_end|>
<|im_start|>chef:
In a large mixing bowl, combine all ingredients except for 1 tablespoon butter; beat well until smooth. Add remaining butter and stir until mixture is lightly coated and creamy.
Time taken for inference: 2.17 seconds


### **Example No:3**

In [40]:
instruction = "Imagine you're a chef, and you've got these delicious ingredients in front of you. Now, all you need are to give the directions on how to cook up a mouthwatering dish."
ingredients = '["1 c. peanut butter", "3/4 c. graham cracker crumbs", "1 c. melted butter", "1 lb. (3 1/2 c.) powdered sugar", "1 large pkg. chocolate chips"]'
generate_response(instruction=instruction, user_input=ingredients)

<|im_start|>user
Imagine you're a chef, and you've got these delicious ingredients in front of you. Now, all you need are to give the directions on how to cook up a mouthwatering dish.<|im_end|>
<|im_start|>input
["1 c. peanut butter", "3/4 c. graham cracker crumbs", "1 c. melted butter", "1 lb. (3 1/2 c.) powdered sugar", "1 large pkg. chocolate chips"]<|im_end|>
<|im_start|>chef:
"Preheat oven to 350\u00b0."<|im_end|>
Time taken for inference: 1.82 seconds


### **Example No:4**

In [42]:
instruction = "Imagine you're a chef, and you've got these delicious ingredients in front of you. Now, all you need are to give the directions on how to cook up a mouthwatering dish."
ingredients = '["1 small jar chipped beef, cut up", "4 boned chicken breasts", "1 can cream of mushroom soup", "1 carton sour cream"]'
generate_response(instruction=instruction, user_input=ingredients)

<|im_start|>user
Imagine you're a chef, and you've got these delicious ingredients in front of you. Now, all you need are to give the directions on how to cook up a mouthwatering dish.<|im_end|>
<|im_start|>input
["1 small jar chipped beef, cut up", "4 boned chicken breasts", "1 can cream of mushroom soup", "1 carton sour cream"]<|im_end|>
<|im_start|>chef:
1. Preheat oven to 350\u00b0.
2. In large bowl, combine beef, chicken and soup; mix well.
3. Add remaining ingredients; stir until thoroughly blended.
4. Pour into ungreased 9x13-inch pan. Bake for about 60 minutes or until internal temperature reaches 170\u00b0 (internal thermometer should read at least 180\u00b0).
Time taken for inference: 3.26 seconds
